In [24]:
import advent
advent.scrape(2015, 19)
data = advent.get_lines_doublenewline(19)
rules_raw, molecule = data
molecule: str = molecule[0]
MAXLEN = len(molecule)
rules = []
for rule in rules_raw:
    rules.append(rule.split(' => '))

In [25]:
# I turned this into a function for part 2, but it's really just the same what I wrote for part 1 just wrapped in a function
# I also added a tuple to possibilities instead of the element, but this has no impact on part 2
def adjacent(molecule: str):
    if len(molecule) > MAXLEN: return []
    possibilities = set([])
    for left, right in rules:
        if left in molecule:
            indexes = [i for i in range(len(molecule)) if molecule.startswith(left, i)]
            for ix in indexes:
                possibilities.add((molecule[:ix] + molecule[ix:].replace(left, right, 1), 1))
    return possibilities
print(len(adjacent(molecule)))


576


In [28]:
# Could try dynamic programming but dijkstra seems more fun :D
from advent.maze import solve_maze

# This didn't end up working
#solve_maze('e', lambda x: x==molecule, adjacent)


In [ ]:
def adjacent_reverse(molecule: str):
    for right, left in rules:
        if left in molecule:
            for ix in range(len(molecule)):
                if molecule.startswith(left, ix):
                    yield (molecule[:ix] + molecule[ix:].replace(left, right, 1), 1)

print(list(adjacent_reverse("SiRnFYFArCaF")))

solve_maze(molecule, lambda x: x == 'e', adjacent_reverse)
               